# Optimización con decenso del gradiente

### Decenso por gradiente

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

np.random.seed(111)

def descenso_por_gradiente(funcion, gradiente_funcion, x_inicial, tasa_aprendizaje=0.01, max_iter=1000, tolerancia=1e-6):
    """
    Optimiza una función objetivo utilizando el método de descenso por gradiente.
    """
    x = np.array(x_inicial, dtype=float)
    for i in range(max_iter):
        gradiente = gradiente_funcion(x)
        norma_gradiente = np.linalg.norm(gradiente)

        if norma_gradiente < tolerancia:
            print(f"Convergencia alcanzada en {i + 1} iteraciones con gradiente {norma_gradiente:.2e}.")
            break

        # Actualización del valor de x
        x -= np.array(tasa_aprendizaje * gradiente, dtype=float)

    else:
        print("Se alcanzó el número máximo de iteraciones sin convergencia.")

    return x, funcion(x), i + 1


def descenso_por_gradiente_con_historial(funcion, gradiente_funcion, x_inicial, tasa_aprendizaje=0.01, max_iter=1000, tolerancia=1e-6):
    x = np.array(x_inicial, dtype=float)
    historial = [x.copy()]  # Registro del historial de puntos
    valores_funcion = [funcion(x)]  # Registro de valores de la función

    for i in range(max_iter):
        gradiente = gradiente_funcion(x)
        max_grad_norm = 1e15
        gradiente = gradiente_funcion(x)
        if np.linalg.norm(gradiente) > max_grad_norm:
          gradiente = gradiente / np.linalg.norm(gradiente) * max_grad_norm
        norma_gradiente = np.linalg.norm(gradiente)

        if norma_gradiente < tolerancia:
            print(f"Convergencia alcanzada en {i + 1} iteraciones con gradiente {norma_gradiente:.2e}.")
            break

        x -= np.array(tasa_aprendizaje * gradiente, dtype=float)
        if i % 10 == 0:  # Guardar cada 10 iteraciones
            historial.append(x.copy())
            valores_funcion.append(funcion(x))

    historial.append(x.copy())  # Guardar el punto final
    valores_funcion.append(funcion(x))

    # Encontrar el punto más pequeño visitado
    min_index = np.argmin(valores_funcion)
    return x, funcion(x), i + 1, historial, min_index



## Rosenbrock

### Definición de la función de Rosenbrock y su gradiente

In [2]:
def funcion_rosenbrock(x, a=1, b=100):
    """Calcula el valor de la función de Rosenbrock para un vector x.

    Args:
      x (array): Vector de entrada (debe ser de tamaño 2).
      a (float): Parámetro de la función (por defecto 1).
      b (float): Parámetro de la función (por defecto 100).

    Retorna:
    float: Valor de la función de Rosenbrock.
    """
    if len(x) < 2:
        raise ValueError("La función de Rosenbrock está implementada solo para al menos 2 dimensiones.")
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2 + sum((a - x[i])**2 + b * (x[i+1] - x[i]**2)**2 for i in range(1, len(x)-1))

def gradiente_rosenbrock(x, a=1, b=100):
    """Calcula el gradiente de la función de Rosenbrock en un punto x.

    Args:
      x (array): Vector de entrada de tamaño n.
      a (float): Parámetro de la función (por defecto 1).
      b (float): Parámetro de la función (por defecto 100).

    Returns:
      array: Gradiente de la función de Rosenbrock.

    """
    n = len(x)
    gradiente = np.zeros_like(x)

    for i in range(n):

        if i == 0:
            gradiente[i] = -2 * (a - x[i]) - 4 * b * x[i] * (x[i + 1] - x[i]**2)
        elif i == n - 1:
            gradiente[i] = 2 * b * (x[i] - x[i - 1]**2)
        else:
            gradiente[i] = -2 * (a - x[i]) - 4 * b * x[i] * (x[i + 1] - x[i]**2) + 2 * b * (x[i] - x[i - 1]**2)

    return gradiente

### Ejecución para 2 y 3 dimensiones

In [3]:
def prueba_con_condiciones_aleatorias_rosenbrock(dimensiones=2):
    x_inicial = np.random.uniform(-5, 5, size=dimensiones)
    print(f"Condición inicial aleatoria: {x_inicial}")

    resultado = descenso_por_gradiente(funcion_rosenbrock, gradiente_rosenbrock, x_inicial, tasa_aprendizaje=0.0001, max_iter=100000000)

    print("Punto óptimo:", resultado[0])
    print("Valor en el óptimo:", resultado[1])
    print("Iteraciones:", resultado[2])

print("prueba para 3 dimensiones")
prueba_con_condiciones_aleatorias_rosenbrock(dimensiones=3)

print("\n")
print("prueba para 2 dimensiones")
prueba_con_condiciones_aleatorias_rosenbrock(dimensiones=2)

prueba para 3 dimensiones
Condición inicial aleatoria: [ 1.12170176 -3.30930246 -0.63940981]
Convergencia alcanzada en 268560 iteraciones con gradiente 1.00e-06.
Punto óptimo: [0.99999954 0.99999908 0.99999816]
Valor en el óptimo: 1.0521959649743357e-12
Iteraciones: 268560


prueba para 2 dimensiones
Condición inicial aleatoria: [ 2.69262473 -2.04674696]
Convergencia alcanzada en 313013 iteraciones con gradiente 1.00e-06.
Punto óptimo: [0.99999888 0.99999776]
Valor en el óptimo: 1.2519569937887262e-12
Iteraciones: 313013


### Grafico en 2d demostrativo

In [4]:
def crear_video_rosenbrock(historial, min_index, dimensiones=2, a=1, b=100, filename="rosenbrock_optim.mp4", fps=30, duracion=10):
    x = np.linspace(-2, 2, 400)
    y = np.linspace(-1, 3, 400)
    X, Y = np.meshgrid(x, y)
    Z = (a - X)**2 + b * (Y - X**2)**2

    # Interpolación para ajustar historial a la duración y los FPS
    total_frames = fps * duracion
    indices = np.linspace(0, len(historial) - 1, total_frames).astype(int)
    historial_interpolado = [historial[i] for i in indices]

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.contourf(X, Y, Z, levels=50, cmap="viridis", alpha=0.6)
    ax.contour(X, Y, Z, levels=10, colors="white", linewidths=0.5)

    # Inicializar puntos
    point, = ax.plot([], [], "ro", markersize=5)  # Punto móvil (rojo)
    min_point, = ax.plot([], [], "go", markersize=10)  # Punto más pequeño dinámico (verde)
    final_point, = ax.plot([], [], "bo", markersize=10)  # Punto final estático (azul)

    # Función de Rosenbrock para evaluar valores de Z
    def rosenbrock_value(x):
        return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

    def actualizar(frame):
        # Actualizar punto móvil (rojo)
        point.set_data(historial_interpolado[frame][0], historial_interpolado[frame][1])

        # Calcular el punto más pequeño dinámicamente (verde)
        valores_actuales = [rosenbrock_value(historial_interpolado[i]) for i in range(frame + 1)]

        # El punto final permanece estático (azul)
        final_point.set_data(1, 1)

        return point, final_point

    anim = FuncAnimation(fig, actualizar, frames=total_frames, interval=1000/fps, blit=True)
    anim.save(filename, writer="pillow", fps=fps)  # Cambiado a "pillow" para guardar GIF
    plt.close(fig)


def prueba_con_condiciones_aleatorias_rosenbrock_video(dimensiones=2, tasa_aprendizaje=0.001):
    x_inicial = np.random.uniform(-5, 5, size=dimensiones)
    print(f"Condición inicial aleatoria: {x_inicial}")

    resultado = descenso_por_gradiente_con_historial(
        funcion_rosenbrock, gradiente_rosenbrock, x_inicial, tasa_aprendizaje=tasa_aprendizaje, max_iter=10000000
    )

    print("Punto óptimo:", resultado[0])
    print("Valor en el óptimo:", resultado[1])
    print("Iteraciones:", resultado[2])

    crear_video_rosenbrock(resultado[3], resultado[4], filename=f"rosenbrock_optim_{tasa_aprendizaje}.gif")
    print("Video generado y guardado como 'rosenbrock_optim.gif'.")

prueba_con_condiciones_aleatorias_rosenbrock_video(tasa_aprendizaje=0.0001)

Condición inicial aleatoria: [-3.50837043 -4.77521675]
Convergencia alcanzada en 313248 iteraciones con gradiente 1.00e-06.
Punto óptimo: [0.99999888 0.99999776]
Valor en el óptimo: 1.2519849928253419e-12
Iteraciones: 313248


<ipython-input-4-f986a609106a>:27: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  point.set_data(historial_interpolado[frame][0], historial_interpolado[frame][1])
<ipython-input-4-f986a609106a>:33: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  final_point.set_data(1, 1)
<ipython-input-4-f986a609106a>:27: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  point.set_data(historial_interpolado[frame][0], historial_interpolado[frame][1])
<ipython-input-4-f986a609106a>:33: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  final_point.set_data(1, 1)


Video generado y guardado como 'rosenbrock_optim.gif'.


## Rastrigin

### Definición de la función de Rastrigin y su gradiente

In [5]:
def funcion_rastrigin(x, A=10):
    """Calcula el valor de la función de Rastrigin para un vector x.

    Args:
      x (array): Vector de entrada.
      A (float): Parámetro de la función (por defecto 10).

    Retorna:
      float: Valor de la función de Rastrigin.
    """
    n = len(x)
    return A * n + np.sum(x**2 - A * np.cos(2 * np.pi * x))

def gradiente_rastrigin(x, A=10):
    """Calcula el gradiente de la función de Rastrigin en un punto x.

    Args:
      x (array): Vector de entrada de tamaño n.
      A (float): Parámetro de la función (por defecto 10).

    Returns:
      array: Gradiente de la función de Rastrigin.
    """
    gradiente = 2 * x + 2 * np.pi * A * np.sin(2 * np.pi * x)
    grad = gradiente + np.random.normal(0, 0.1, size=x.shape)
    return grad



### Ejecución para 2 y 3 dimensiones

In [6]:
def prueba_con_condiciones_aleatorias_rastrigin(dimensiones=2, tasa_aprendizaje=0.01, max_iter=10000000):
    """Prueba el descenso por gradiente en la función de Rastrigin con condiciones iniciales aleatorias.

    Args:
      dimensiones (int): Número de dimensiones de la función (por defecto 2).

    Retorna:
      None: Imprime el resultado del descenso por gradiente.
    """
    x_inicial = np.random.uniform(-5.12, 5.12, size=dimensiones)
    print(f"Condición inicial aleatoria: {x_inicial}")

    resultado = descenso_por_gradiente(funcion_rastrigin, gradiente_rastrigin, x_inicial, tasa_aprendizaje=0.1, max_iter=1000000)

    print("Punto óptimo:", resultado[0])
    print("Valor en el óptimo:", resultado[1])
    print("Iteraciones:", resultado[2])

print("prueba para 3 dimensiones")
prueba_con_condiciones_aleatorias_rastrigin(dimensiones=3)

print("\nprueba para 2 dimensiones")
prueba_con_condiciones_aleatorias_rastrigin(dimensiones=2)

prueba para 3 dimensiones
Condición inicial aleatoria: [-0.8169012  -2.67589487 -1.6624006 ]
Se alcanzó el número máximo de iteraciones sin convergencia.
Punto óptimo: [-17.72910772  11.68829462 -15.69440587]
Valor en el óptimo: 735.7639401106309
Iteraciones: 1000000

prueba para 2 dimensiones
Condición inicial aleatoria: [-1.43449338  2.39157752]
Se alcanzó el número máximo de iteraciones sin convergencia.
Punto óptimo: [ 2.48313895 -0.90610953]
Valor en el óptimo: 28.621159076720424
Iteraciones: 1000000


### Creación de ilustración en 2d

In [8]:
def crear_video_rastrigin(historial, min_index, dimensiones=2, A=10, filename="rastrigin_optim.mp4", fps=30, duracion=10):
    x = np.linspace(-5.12, 5.12, 400)
    y = np.linspace(-5.12, 5.12, 400)
    X, Y = np.meshgrid(x, y)
    Z = A * len(X) + (X**2 - A * np.cos(2 * np.pi * X)) + (Y**2 - A * np.cos(2 * np.pi * Y))

    # Interpolación para ajustar historial a la duración y los FPS
    total_frames = fps * duracion
    indices = np.linspace(0, len(historial) - 1, total_frames).astype(int)
    historial_interpolado = [historial[i] for i in indices]

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.contourf(X, Y, Z, levels=50, cmap="viridis", alpha=0.6)
    ax.contour(X, Y, Z, levels=10, colors="white", linewidths=0.5)

    # Inicializar puntos
    point, = ax.plot([], [], "ro", markersize=5)  # Punto móvil (rojo)
    min_point, = ax.plot([], [], "go", markersize=10)  # Punto más pequeño dinámico (verde)
    final_point, = ax.plot([], [], "bo", markersize=10)  # Punto final estático (azul)

    # Función de Rastrigin para evaluar valores de Z
    def rastrigin_value(x):
        return A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x))

    def actualizar(frame):
        # Actualizar punto móvil (rojo)
        point.set_data(historial_interpolado[frame][0], historial_interpolado[frame][1])

        # Calcular el punto más pequeño dinámicamente (verde)
        valores_actuales = [rastrigin_value(historial_interpolado[i]) for i in range(frame + 1)]

        # El punto final permanece estático (azul)
        final_point.set_data(0, 0)

        return point, final_point

    anim = FuncAnimation(fig, actualizar, frames=total_frames, interval=1000/fps, blit=True)
    anim.save(filename, writer="pillow", fps=fps)  # Cambiado a "pillow" para guardar GIF
    plt.close(fig)


def prueba_con_condiciones_aleatorias_rastrigin_video(dimensiones=2, tasa_aprendizaje=0.01, max_iter=1000000):
    x_inicial = np.random.uniform(-5.12, 5.12, size=dimensiones)
    print(f"Condición inicial aleatoria: {x_inicial}")

    resultado = descenso_por_gradiente_con_historial(
        funcion_rastrigin, gradiente_rastrigin, x_inicial, tasa_aprendizaje=tasa_aprendizaje, max_iter=max_iter
    )

    print("Punto óptimo:", resultado[0])
    print("Valor en el óptimo:", resultado[1])
    print("Iteraciones:", resultado[2])

    crear_video_rastrigin(resultado[3], resultado[4], filename=f"rastrigin_optim_{tasa_aprendizaje}.gif")
    print("Video generado y guardado como 'rastrigin_optim.gif'.")

prueba_con_condiciones_aleatorias_rastrigin_video()

Condición inicial aleatoria: [-4.78426826  3.18296741]
Punto óptimo: [-2.70030128  2.54803406]
Valor en el óptimo: 46.404275132306815
Iteraciones: 1000000


<ipython-input-8-b257bfb4b9b1>:27: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  point.set_data(historial_interpolado[frame][0], historial_interpolado[frame][1])
<ipython-input-8-b257bfb4b9b1>:33: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  final_point.set_data(0, 0)
<ipython-input-8-b257bfb4b9b1>:27: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  point.set_data(historial_interpolado[frame][0], historial_interpolado[frame][1])
<ipython-input-8-b257bfb4b9b1>:33: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  final_point.set_data(0, 0)


Video generado y guardado como 'rastrigin_optim.gif'.


# Algoritmos Evolutivos

## Rosenbrock

### Evolución diferencial

In [9]:
from scipy.optimize import differential_evolution

def funcion_rosenbrock(x, a=1, b=100):
    """Calcula el valor de la función de Rosenbrock para un vector x.

    Args:
      x (array): Vector de entrada (debe ser de tamaño 2).
      a (float): Parámetro de la función (por defecto 1).
      b (float): Parámetro de la función (por defecto 100).

    Retorna:
    float: Valor de la función de Rosenbrock.
    """
    if len(x) < 2:
        raise ValueError("La función de Rosenbrock está implementada solo para al menos 2 dimensiones.")
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2 + sum((a - x[i])**2 + b * (x[i+1] - x[i]**2)**2 for i in range(1, len(x)-1))

# Definimos el rango de búsqueda para cada dimensión
bounds_2d = [(-5, 5), (-5, 5)]
bounds_3d = [(-5, 5), (-5, 5), (-5, 5)]

# Optimización en 2 dimensiones
result_2d = differential_evolution(funcion_rosenbrock, bounds_2d)
print("Resultado DE (2D):", result_2d)

# Optimización en 3 dimensiones
result_3d = differential_evolution(funcion_rosenbrock, bounds_3d)
print("Resultado DE (3D):", result_3d)

Resultado DE (2D):              message: Optimization terminated successfully.
             success: True
                 fun: 4.979684464207637e-30
                   x: [ 1.000e+00  1.000e+00]
                 nit: 126
                nfev: 3813
          population: [[ 1.000e+00  1.000e+00]
                       [ 1.000e+00  1.000e+00]
                       ...
                       [ 1.000e+00  1.000e+00]
                       [ 1.000e+00  1.000e+00]]
 population_energies: [ 4.980e-30  4.980e-30 ...  4.980e-30  4.980e-30]
Resultado DE (3D):              message: Optimization terminated successfully.
             success: True
                 fun: 9.959368928415274e-30
                   x: [ 1.000e+00  1.000e+00  1.000e+00]
                 nit: 244
                nfev: 11029
          population: [[ 1.000e+00  1.000e+00  1.000e+00]
                       [ 1.000e+00  1.000e+00  1.000e+00]
                       ...
                       [ 1.000e+00  1.000e+00  1.000e+00]
 

### Optimización de particulas

In [10]:
import pyswarms as ps

# Definimos la función objetivo
def rosenbrock_pso(x):
    return np.apply_along_axis(funcion_rosenbrock, 1, x)

# Configuración para 2D y 3D
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}  # Coeficientes del algoritmo PSO
bounds_2d = (np.array([-5, -5]), np.array([5, 5]))
bounds_3d = (np.array([-5, -5, -5]), np.array([5, 5, 5]))

# PSO para 2D
optimizer_2d = ps.single.GlobalBestPSO(n_particles=50, dimensions=2, options=options, bounds=bounds_2d)
best_cost_2d, best_pos_2d = optimizer_2d.optimize(rosenbrock_pso, iters=100)
print("Resultado PSO (2D):", best_cost_2d, best_pos_2d)

# PSO para 3D
optimizer_3d = ps.single.GlobalBestPSO(n_particles=50, dimensions=3, options=options, bounds=bounds_3d)
best_cost_3d, best_pos_3d = optimizer_3d.optimize(rosenbrock_pso, iters=1000)
print("Resultado PSO (3D):", best_cost_3d, best_pos_3d)

2024-11-30 04:47:31,803 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=5.02e-5
2024-11-30 04:47:32,071 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 5.016486241380026e-05, best pos: [1.00705817 1.01422508]
2024-11-30 04:47:32,105 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Resultado PSO (2D): 5.016486241380026e-05 [1.00705817 1.01422508]


pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=2.22e-7
2024-11-30 04:47:39,117 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 2.2191093574781943e-07, best pos: [1.00020893 1.00042098 1.00084119]


Resultado PSO (3D): 2.2191093574781943e-07 [1.00020893 1.00042098 1.00084119]


### Algoritmo evolutivo

In [11]:
from deap import base, creator, tools, algorithms
import random

# Configuración de la función objetivo con DEAP
def rosenbrock_deap(individual):
    """Función objetivo para DEAP, adaptada para evaluar un individuo."""
    return (funcion_rosenbrock(individual),)  # Regresamos una tupla porque DEAP espera múltiples objetivos

# Crear clases para el problema
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar la función
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Crear individuos y población
toolbox.register("attr_float", random.uniform, -5, 5)  # Rango de búsqueda [-5, 5]
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)  # Tamaño del individuo = 2 (para 2 dimensiones)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operadores genéticos
toolbox.register("evaluate", rosenbrock_deap)
toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Cruce (Blend crossover)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)  # Mutación
toolbox.register("select", tools.selTournament, tournsize=3)  # Selección por torneo

# Algoritmo Genético
def run_ga(dimensions=2, population_size=100, generations=100):
    """Ejecuta el algoritmo genético con DEAP."""
    # Crear la población inicial
    population = toolbox.population(n=population_size)

    # Ejecutar el algoritmo genético
    result, log = algorithms.eaSimple(
        population,
        toolbox,
        cxpb=0.6,  # Reduce ligeramente cruce
        mutpb=0.4,  # Aumenta mutación
        ngen=300,   # Más generaciones
        verbose=False
    )

    # Seleccionar el mejor individuo
    best_individual = tools.selBest(population, k=1)[0]
    return best_individual, funcion_rosenbrock(best_individual)

# Ejecutar para 2 dimensiones]
best_ind_2d, best_cost_2d = run_ga(dimensions=2, population_size=300, generations=500)
print("Resultado GA (2D):", best_ind_2d, "Costo:", best_cost_2d)

Resultado GA (2D): [0.9836535371335493, 0.9675140156538657] Costo: 0.000267570040828956


## Rastringin



### Evolución diferencial

In [12]:
def funcion_rastrigin(x, A=10):
    """Calcula el valor de la función de Rastrigin para un vector x.

    Args:
      x (array): Vector de entrada.
      A (float): Parámetro de la función (por defecto 10).

    Retorna:
      float: Valor de la función de Rastrigin.
    """
    n = len(x)
    return A * n + np.sum(x**2 - A * np.cos(2 * np.pi * x))

# Definimos el rango de búsqueda para cada dimensión
bounds_2d = [(-5.12, 5.12), (-5.12, 5.12)]
bounds_3d = [(-5.12, 5.12), (-5.12, 5.12), (-5.12, 5.12)]

# Optimización en 2 dimensiones
result_2d = differential_evolution(funcion_rastrigin, bounds_2d)
print("Resultado DE (2D):", result_2d)

# Optimización en 3 dimensiones
result_3d = differential_evolution(funcion_rastrigin, bounds_3d)
print("Resultado DE (3D):", result_3d)

Resultado DE (2D):              message: Optimization terminated successfully.
             success: True
                 fun: 0.0
                   x: [ 2.519e-09 -1.156e-09]
                 nit: 65
                nfev: 1983
          population: [[ 2.519e-09 -1.156e-09]
                       [-1.151e-09 -3.121e-09]
                       ...
                       [ 2.329e-09 -1.417e-10]
                       [ 2.915e-10 -8.283e-10]]
 population_energies: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
Resultado DE (3D):              message: Optimization terminated successfully.
             success: True
                 fun: 0.0
                   x: [-6.893e-10 -9.393e-10 -8.071e-10]
                 nit: 93
                nfev: 4234
          population: [[-6.893e-10 -9.393e-10 -8.071e-10]
                       [-1.268e-09 -1.103e-09 -2.266e-09]
                       ...
                       [ 3.571e-09 -1.328e-09 -1.229e-09]
                       [ 1.044e-09 -2.65

### Optimización de particulas

In [13]:
import pyswarms as ps

# Definimos la función objetivo
def rastringin_pso(x):
    return np.apply_along_axis(funcion_rastrigin, 1, x)

# Configuración para 2D y 3D
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}  # Coeficientes del algoritmo PSO
bounds_2d = (np.array([-5.12, -5.12]), np.array([5.12, 5.12]))
bounds_3d = (np.array([-5.12, -5.12, -5.12]), np.array([5.12, 5.12, 5.12]))

# PSO para 2D
optimizer_2d = ps.single.GlobalBestPSO(n_particles=100, dimensions=2, options=options, bounds=bounds_2d)
best_cost_2d, best_pos_2d = optimizer_2d.optimize(rastringin_pso, iters=100)
print("Resultado PSO (2D):", best_cost_2d, best_pos_2d)

# PSO para 3D
optimizer_3d = ps.single.GlobalBestPSO(n_particles=100, dimensions=3, options=options, bounds=bounds_3d)
best_cost_3d, best_pos_3d = optimizer_3d.optimize(rastringin_pso, iters=1000)
print("Resultado PSO (3D):", best_cost_3d, best_pos_3d)

2024-11-30 04:47:45,252 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=5.66e-7
2024-11-30 04:47:45,717 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 5.661129947043264e-07, best pos: [2.14910808e-06 5.33749686e-05]
2024-11-30 04:47:45,730 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Resultado PSO (2D): 5.661129947043264e-07 [2.14910808e-06 5.33749686e-05]


pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0
2024-11-30 04:47:51,253 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [ 1.67117283e-09 -3.56071624e-09  6.12646323e-10]


Resultado PSO (3D): 0.0 [ 1.67117283e-09 -3.56071624e-09  6.12646323e-10]


### Algoritmo Evolutivo

In [14]:
import numpy as np
from deap import base, creator, tools, algorithms
import random

# Función de Rastrigin
def funcion_rastrigin(x, A=10):
    """Calcula el valor de la función de Rastrigin para un vector x.

    Args:
      x (array): Vector de entrada.
      A (float): Parámetro de la función (por defecto 10).

    Retorna:
      float: Valor de la función de Rastrigin.
    """
    x = np.array(x)
    n = len(x)
    return A * n + np.sum(x**2 - A * np.cos(2 * np.pi * x))

# Configuración de la función objetivo con DEAP
def rastrigin_deap(individual):
    """Función objetivo para DEAP, adaptada para evaluar un individuo."""
    return (funcion_rastrigin(individual),)  # Regresamos una tupla porque DEAP espera múltiples objetivos

# Crear clases para el problema
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar la función
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Crear individuos y población
toolbox.register("attr_float", random.uniform, -5.12, 5.12)  # Rango de búsqueda [-5, 5]
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)  # Tamaño del individuo = 2 (para 2 dimensiones)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operadores genéticos
toolbox.register("evaluate", rastrigin_deap)
toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Cruce (Blend crossover)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)  # Mutación
toolbox.register("select", tools.selTournament, tournsize=3)  # Selección por torneo


# Algoritmo Genético
def run_ga(dimensions=2, population_size=1000, generations=1000):
    """Ejecuta el algoritmo genético con DEAP."""
    # Crear la población inicial
    population = toolbox.population(n=population_size)

    # Ejecutar el algoritmo genético
    result, log = algorithms.eaSimple(
        population,
        toolbox,
        cxpb=0.6,  # Reduce ligeramente cruce
        mutpb=0.4,  # Aumenta mutación
        ngen=300,   # Más generaciones
        verbose=False
    )

    # Seleccionar el mejor individuo
    best_individual = tools.selBest(population, k=1)[0]
    return best_individual, funcion_rastrigin(best_individual)

# Ejecutar para 2 dimensiones
best_ind_2d, best_cost_2d = run_ga(dimensions=2, population_size=300, generations=500)
print("Resultado GA (2D):", np.round(best_ind_2d, 3), "Costo:", best_cost_2d)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Resultado GA (2D): [-0. -0.] Costo: 0.0
